# Electoral College Data Mapping

The purpose of this project is to analyze the how the electoral college votes were distributed after the 2010 census, look at how the redistribution of population (by estimate) has shifted over time until today, and what that means for the voter per electoral college vote in each state. I will also analyze how we expect the electoral college to be redistributed after the 2020 census, given census bureau predictions.

An analysis will also be performed on the percentage likelihood of each state giving its electoral college votes to a particular party and their respective nominees, based purely on historical data. The aim is to demonstrate which states have the greatest power per vote, given both their current electoral votes allotted and the likelihood of that state assigning thier votes to either candidate.

In [16]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import json

import sqlite3
%matplotlib inline

In [2]:
#define how to retrieve api keys

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [3]:
#get key for census bureau api
keys = get_keys("/Users/flatironschool/.secret/census_api.json")

api_key = keys['api_key']

In [5]:
#make and print request for census count and estimates from 2010-2019 for all states
year = '2019'

url = 'https://api.census.gov/data/{}/pep/population'.format(year)

variables = ['DATE_CODE',
             'DATE_DESC',
             'POP',
             'NAME']

granularity = 'state:*'

params = {'get': ','.join(variables), 'for': granularity, 'key': api_key}

r = requests.get(url, params=params)
print(r.url)
print(r)
print(type(r.text))
print(r.text[:1000])

https://api.census.gov/data/2019/pep/population?get=DATE_CODE%2CDATE_DESC%2CPOP%2CNAME&for=state%3A%2A&key=b7961d22ec04ff1777be8a0450921d3f28af8315
<Response [200]>
<class 'str'>
[["DATE_CODE","DATE_DESC","POP","NAME","state"],
["1","4/1/2010 Census population","5303925","Minnesota","27"],
["2","4/1/2010 population estimates base","5303927","Minnesota","27"],
["3","7/1/2010 population estimate","5310828","Minnesota","27"],
["4","7/1/2011 population estimate","5346143","Minnesota","27"],
["5","7/1/2012 population estimate","5376643","Minnesota","27"],
["6","7/1/2013 population estimate","5413479","Minnesota","27"],
["7","7/1/2014 population estimate","5451079","Minnesota","27"],
["8","7/1/2015 population estimate","5482032","Minnesota","27"],
["9","7/1/2016 population estimate","5522744","Minnesota","27"],
["10","7/1/2017 population estimate","5566230","Minnesota","27"],
["11","7/1/2018 population estimate","5606249","Minnesota","27"],
["12","7/1/2019 population estimate","5639632","Min

In [26]:
#clean data (only need population, state, and datetime) with pandas before putting into SQL database
data = r.json()
df = pd.DataFrame(data[1:], columns=data[0])
df['DATETIME'] = df.DATE_DESC.apply(lambda x: pd.to_datetime(x[:9]))
df.drop(df[df.DATE_CODE == '2'].index, inplace=True)
df.drop(['state','DATE_CODE', 'DATE_DESC'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df['POP'] = df.POP.astype('int64')
df = df[['DATETIME', 'NAME', 'POP']]

display(df.head(15))
display(df.info())

,DATETIME,NAME,POP
0,2010-04-01,Minnesota,5303925
1,2010-07-01,Minnesota,5310828
2,2011-07-01,Minnesota,5346143
3,2012-07-01,Minnesota,5376643
4,2013-07-01,Minnesota,5413479
5,2014-07-01,Minnesota,5451079
6,2015-07-01,Minnesota,5482032
7,2016-07-01,Minnesota,5522744
8,2017-07-01,Minnesota,5566230
9,2018-07-01,Minnesota,5606249


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 3 columns):
DATETIME    572 non-null datetime64[ns]
NAME        572 non-null object
POP         572 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 13.5+ KB


None

In [27]:
#put pop data into SQL database for easy retrieval
conn = sqlite3.connect('census_pop_data.db')
c = conn.cursor()

In [25]:
c.execute('CREATE TABLE POPULATION (Datetime text, State text, Population integer)')
conn.commit()

In [28]:
df.to_sql('POPULATION', conn, if_exists='replace')

In [29]:
c.execute('SELECT * FROM population')
for row in c.fetchall():
    print(row)

(0, '2010-04-01 00:00:00', 'Minnesota', 5303925)
(1, '2010-07-01 00:00:00', 'Minnesota', 5310828)
(2, '2011-07-01 00:00:00', 'Minnesota', 5346143)
(3, '2012-07-01 00:00:00', 'Minnesota', 5376643)
(4, '2013-07-01 00:00:00', 'Minnesota', 5413479)
(5, '2014-07-01 00:00:00', 'Minnesota', 5451079)
(6, '2015-07-01 00:00:00', 'Minnesota', 5482032)
(7, '2016-07-01 00:00:00', 'Minnesota', 5522744)
(8, '2017-07-01 00:00:00', 'Minnesota', 5566230)
(9, '2018-07-01 00:00:00', 'Minnesota', 5606249)
(10, '2019-07-01 00:00:00', 'Minnesota', 5639632)
(11, '2010-04-01 00:00:00', 'Mississippi', 2967297)
(12, '2010-07-01 00:00:00', 'Mississippi', 2970548)
(13, '2011-07-01 00:00:00', 'Mississippi', 2978731)
(14, '2012-07-01 00:00:00', 'Mississippi', 2983816)
(15, '2013-07-01 00:00:00', 'Mississippi', 2988711)
(16, '2014-07-01 00:00:00', 'Mississippi', 2990468)
(17, '2015-07-01 00:00:00', 'Mississippi', 2988471)
(18, '2016-07-01 00:00:00', 'Mississippi', 2987938)
(19, '2017-07-01 00:00:00', 'Mississippi', 2

# Working Zone